# Keras - Tensorboard
In this lab you will use Tensorboard functionalities with a Keras model with Tensorflow backend

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, SGD
import os
import numpy as np
import time

### Load Cifar10 data

In [2]:
def parse_tfrecord(example):
    feature={'idx'     : tf.FixedLenFeature((), tf.int64),
             'label'   : tf.FixedLenFeature((), tf.int64),
             'image'   : tf.FixedLenFeature((), tf.string, default_value="")}
    parsed = tf.parse_single_example(example, feature)
    image = tf.decode_raw(parsed['image'],tf.float64)
    image = tf.cast(image,tf.float32)
    image = tf.reshape(image,[32,32,3])
    return image, parsed['label']

def image_scaling(x):
    return tf.image.per_image_standardization(x)

def distort(x):
    x = tf.image.resize_image_with_crop_or_pad(x, 40, 40)
    x = tf.random_crop(x, [32, 32, 3])
    x = tf.image.random_flip_left_right(x)
    return x

In [3]:
def dataset_input_fn(params):
    dataset = tf.data.TFRecordDataset(
        params['filenames'],num_parallel_reads=params['threads'])
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=params['threads'])
    dataset = dataset.map(lambda x,y: (image_scaling(x),y),num_parallel_calls=params['threads'])
    if params['mode']==tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.map(lambda x,y: (distort(x),y),num_parallel_calls=params['threads'])
        dataset = dataset.shuffle(buffer_size=params['shuffle_buff'])
    dataset = dataset.repeat()
    dataset = dataset.batch(params['batch'])
    dataset = dataset.prefetch(8*params['batch'])
    return dataset

In [4]:
train_files = !ls ./data/cifar10_data_00*.tfrecords
val_files   = !ls ./data/cifar10_data_01*.tfrecords

train_params = {'filenames'    : train_files,
                'mode'         : tf.estimator.ModeKeys.TRAIN,
                'threads'      : 16,
                'shuffle_buff' : 100000,
                'batch'        : 100
               }

eval_params  = {'filenames'    : val_files,
                'mode'         : tf.estimator.ModeKeys.EVAL,
                'threads'      : 8,
                'batch'        : 200
               }

train_spec = tf.estimator.TrainSpec(input_fn=lambda: dataset_input_fn(train_params),max_steps=200000)
eval_spec  = tf.estimator.EvalSpec(input_fn=lambda: dataset_input_fn(eval_params),steps=10,throttle_secs=60)


### Setup Keras model

In [5]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

In [6]:
#Define Keras model
model = Sequential()
model.add(Conv2D(128, (5, 5), padding='same',
                 input_shape=[32, 32, 3]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model.add(Conv2D(128, (5,5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Dropout(0.2))
model.add(Activation('softmax'))


opt = SGD(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt)
'''
              optimizer=opt,
              metrics=['accuracy'])

'''

"\n              optimizer=opt,\n              metrics=['accuracy'])\n\n"

In [7]:
distribution = tf.contrib.distribute.MirroredStrategy(num_gpus=4)
config = tf.estimator.RunConfig(save_checkpoints_secs = 300,
                                keep_checkpoint_max = 5,
                                session_config=tf.ConfigProto(
                                    allow_soft_placement=True, log_device_placement=True),
                                train_distribute = distribution)

In [8]:
model_params  = {'drop_out'      : 0.2,
                 'dense_units'   : 1024,
                 'learning_rate' : 1e-3,
                 'log'           : True
                }

In [9]:
name = 'cnn_model/keras_model_'
name = name + 'dense' + str(model_params['dense_units']) + '_'
name = name + 'drop' + str(model_params['drop_out']) + '_'
name = name + 'lr' + str(model_params['learning_rate']) + '_'
name = name + time.strftime("%Y%m%d%H%M%S")
model_dir  = os.path.join('./',name)

print(model_dir)

./cnn_model/keras_model_dense1024_drop0.2_lr0.001_20180722081706


In [19]:
estimator = 0

In [10]:
estimator = keras.estimator.model_to_estimator(
  keras_model=model,
  config=config,
  model_dir=model_dir)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_service': None, '_task_id': 0, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7fd30907a6a0>, '_session_config': allow_soft_placement: true
log_device_placement: true
, '_save_summary_steps': 100, '_task_type': 'worker', '_num_worker_replicas': 1, '_is_chief': True, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_tf_random_seed': None, '_device_fn': None, '_evaluation_master': '', '_master': '', '_save_checkpoints_secs': 300, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './cnn_model/keras_model_dense1024_drop0.2_lr0.001_20180722081706', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd30907ae80>, '_keep_checkpoint_max': 5}


In [11]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Error reported to Coordinator: Use DistributionStrategy.update() to modify a MirroredVariable.
Traceback (most recent call last):
  File "/home/tsaikevin/env/tf18p3/lib/python3.5/site-packages/tensorflow/python/training/coordinator.py", line 297, in stop_on_exception
    yield
  File "/home/tsaikevin/env/tf18p3/lib/python3.5/site-packages/tensorflow/contrib/distribute/python/mirrored_strategy.py", line 480, in run
    self.done = True
  File "/usr/lib/python3.5/contextlib.py", line 77, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/tsaikevin/env/tf18p3/lib/python3.5/site-package

RuntimeError: Use DistributionStrategy.update() to modify a MirroredVariable.

In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

### Setup output for Tensorboard

In [7]:
#Include dependencies to support Tensorboard callback
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard

In [8]:
#Configure Tensorboard callback
tensorboard = TensorBoard(log_dir='./logs/tb-keras', histogram_freq=1, 
                          batch_size=32, write_graph=True, write_grads=True, write_images=False)

In [ ]:
#Run training with callback for Tensorboard
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs, validation_data=(x_test, y_test),
              shuffle=True,
         callbacks = [tensorboard])